<a href="https://colab.research.google.com/github/hsnam95/class2022Fall/blob/main/YES24API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import json
import requests
from bs4 import BeautifulSoup
import re
import numpy as np


In [ ]:
bookID = '176787'

In [ ]:
url = f'http://www.yes24.com/Product/communityModules/GoodsReviewList/{bookID}?Sort=1&PageNumber=1&Type=ALL'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
tmp = soup.find(string=re.compile('reviewCountText'))
tmp = str(tmp)
cnt = re.findall('(?<= reviewCountText = \')\d*(?=\')', tmp)
cnt = int(cnt[0])

nPage = int(np.ceil(cnt/5))

In [ ]:
data = {'title': [],
        'ID': [],
        'date': [],
        'contRating': [],
        'designRating': [],
        'text': []}
df = pd.DataFrame(data)

In [ ]:
i = 0
for n in range(nPage):
  PageNumber = n + 1
  url = f'http://www.yes24.com/Product/communityModules/GoodsReviewList/{bookID}?Sort=1&PageNumber={PageNumber}&Type=ALL'
  response = requests.get(url)
  
  soup = BeautifulSoup(response.content, 'html.parser')
  title = soup.select('span.txt')
  ID = soup.select('a.lnk_id')
  date = soup.select('em.txt_date')
  rating = soup.select('span.review_rating > span.rating');
  contRating = rating[0:9:2]; designRating = rating[1:10:2]
  text = soup.select('div.reviewInfoBot.origin > div.review_cont')

  for r in zip(title, ID, date, contRating, designRating, text):
    title, ID, date, contRating, designRating, text = [r[0].get_text(), r[1].get_text(), r[2].get_text(), r[3].get_text(), r[4].get_text(), r[5].get_text()]    
    row = [title, ID, date, contRating, designRating, text]
    df.loc[i, :] = row
    i +=1

In [ ]:
import xlwt
df.to_excel('review.xls')